<a href="https://colab.research.google.com/github/Waranit-tec/Predict-Project/blob/main/Project_Data_Visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping

import library ที่เราต้องการใช้

In [ ]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import os
from matplotlib import font_manager as fm, rcParams
from matplotlib.ticker import MaxNLocator

ใช้คำสั่ง wget ดาวน์โหลดไฟล์จากเว็บไซต์เพื่อดาวน์โหลดฟอนต์ thsarabunnew

In [ ]:
!wget -q https://github.com/Phonbopit/sarabun-webfont/raw/master/fonts/thsarabunnew-webfont.ttf

สร้าง list ของส่วนหน้าและส่วนหลังของ url เพื่อให้ดึงข้อมูลมาใช้

In [ ]:
main_url = 'https://sgulyano.github.io/eval/'
html_url = ['02226005.html',
            '08846113.html',
            '19689783.html',
            '24363682.html',
            '24674969.html',
            '26742703.html',
            '33472214.html',
            '33561138.html',
            '33565025.html',
            '41557226.html',
            '43951333.html',
            '44868136.html',
            '45435188.html',
            '48033334.html',
            '51435303.html',
            '53231654.html',
            '60189297.html',
            '61116032.html',
            '66887754.html',
            '68346670.html',
            '71728345.html',
            '79550445.html',
            '83255004.html',
            '94026030.html',
            '95860997.html',
            '96905621.html']
eval_url = [main_url + url for url in html_url]

สร้าง function เพื่อดึงข้อมูลจากตารางของแต่ละ html

In [ ]:
#สร้างตารางไว้ใส่ข้อมูล
def get_table_data(item):
    #สร้าง list ว่างที่ชื่อ data เพื่อรอนำข้อมูลมาใส่ใน list
    data = []
    #สร้างคำสั่งให้หา items ที่ชื่อ tbody(ข้อมูลในตาราง))
    table_body = item.find("tbody")
    #สั่งให้หา tr ทั้งหมดใน table_body (ข้อมูลในแถว)
    rows = table_body.find_all("tr")
    #ใช้ฟังก์ชัน find_all เพื่อค้นหา tag ทั้งหมด และนำมาเก็บไว้ใน list
    for row in rows :
        cols = row.find_all(["td","th"])
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele])
    return data

กระบวนการดึงข้อมูลจากเว็บไซต์ทั้งหมด เพื่อนำมาสร้างตาราง

In [ ]:
dfs = [] #สร้าง list ว่าง เพื่อรอนำข้อมูลมาใส่ใน list
for i in range(26): #ใช้คำสั่ง for loop ในการวนซ้ำคำสั่งการดึงข้อมูลจาก 26 เว็บไซต์
    page = i #กำหนดตัวแปร page = i
    r = requests.get(eval_url[page]) #ใช้คำสั่ง requests.get เพื่อดึงข้อมูลจากเว็บไซต์
    soup = BeautifulSoup(r.content,"html.parser") #แปลงเป็น BeautifulSoup object ทำหน้าที่เป็นตัวแทนข้อมูลบนเว็บ และ ใช้คำสั่ง html.parser เพื่ออ่าน code  ทีละบรรทัด
    tables = soup.find_all("table") #ใช้คำสั่ง soup.find_all("table") เพื่อค้นหาตารางทั้งหมดในเว็บไซต์นั้นๆ

    head = pd.DataFrame(get_table_data(tables[0])) #ใช้ฟังก์ชั่นเรียกข้อมูลจากตารางที่ 0 แล้วนำไปเก็บในตัวแปร head
    #เรียกข้อมูลจากตัวแปร head แถวที่ i คอลัมน์ที่ j ; [i,j] แล้วนำไปเก็บในตัวแปรต่างๆ
    subject = head.iloc[0,1]
    semester = head.iloc[2,1] 
    section = head.iloc[1,1]
    N = head.iloc[4,1]
    n = head.iloc[5,1]
    
    df = pd.DataFrame(get_table_data(tables[2])) #สร้าง DataFrame โดยใช้ข้อมูลจากตารางที่ 2 ในแต่ละเว็บไซต์
    df.dropna(how="all",inplace=True) #ใช้ฟังก์ชั่น.dropna() เพื่อลบทุก rows ที่มีค่า NaN
    df.set_index(0) #จัดเรียงข้อมูล
    df.columns = df.iloc[1,:] #ใช้ฟังก์ชั่น .iloc เพื่อเรียกข้อมูลส่วนหัว ของ dataframe
    rows = [0,1] 
    df.drop(rows,axis=0,inplace=True) #ใช้ฟังก์ชัน .drop()เพื่อลบคอลัมน์ ในตัวแปร rows

    #เปลี่ยน data type จาก object เป็น float หรือ int เพื่อนำข้อมูลที่เป็นตัวเลขไปสร้างกราฟ
    for i in range(2,16): #ใช้คำสั่ง for loop ในการวนซ้ำคำสั่ง
        df.iloc[:,i] = pd.to_numeric(df.iloc[:,i])
        
    #สร้างคอลัมน์ "ที่"
    df.set_index("ที่",inplace=True)
    #เอาแถว รวม และ รวมทั้งหมด ออก
    df.drop(df[df.index.str.match("รวม")].index,inplace=True)
    #แยก section
    df["Topic"] = " "
    sec = " "
    c = 0
    for i in df.index:
        if len(i)>3:
            sec = i 
            c += 1
        else:
            df.iloc[c,15] = sec
            c += 1
    df.dropna(how= "any" , inplace= True)

    #การเปลี่ยนชื่อจากร้อยละ เป็น เปอร์เซ็นต์(%)
    cols = []
    count = 5
    for column in df.columns: #ใช้ for loop ในการรัน columns
        if column == 'ร้อยละ':
            cols.append(f'{count}(%)') #เก็บ (f'{count}(%)')  ไว้ในตัวแปร cols
            count-=1
            continue
        cols.append(column) #เก็บ column ไว้ในตัวแปร cols
    df.columns = cols

    #เพิ่มแถวข้อมูลจากตัวแปรต่างๆ ไว้ทางด้านขวาของ DataFrame
    df["section"] = section
    df["รายวิชา"] = subject
    df["ภาคการศึกษา"] = semester
    df["จำนวนนักศึกษาที่จดทะเบียน"] = N
    df["จำนวนนักศึกษาที่ประเมิน"] = n
    #เปลี่ยน data type คอลัมน์ที่ 19 จาก object เป็น float หรือ int เพื่อนำข้อมูลที่เป็นตัวเลขไปสร้างกราฟ
    df.iloc[:,19] = pd.to_numeric(df.iloc[:,19])
    #นำ df ทั้งหมด ไปเก็บไว้ที่ list ของ dfs
    dfs.append(df)
    
#นำข้อมูลจาก 26 เว็บไซต์มาเรียงต่อกัน โดยใช้คำสั่ง pd.concat 
df = pd.concat(dfs)
df

,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,0,0(%),X-Bar,SD,Topic,section,รายวิชา,ภาคการศึกษา,จำนวนนักศึกษาที่จดทะเบียน,จำนวนนักศึกษาที่ประเมิน
ที่,,,,,,,,,,,,,,,,,,,,,
1,สอนอย่างเป็นระบบ,4.0,66.67,1.0,16.67,0.0,0.00,1.0,16.67,0.0,...,0.0,0.0,4.33,1.21,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
2,สอนให้คิด วิเคราะห์ วิจารณ์,4.0,66.67,0.0,0.00,1.0,16.67,1.0,16.67,0.0,...,0.0,0.0,4.17,1.33,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
3,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2.0,33.33,1.0,16.67,1.0,16.67,1.0,16.67,1.0,...,0.0,0.0,3.33,1.63,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
4,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3.0,50.00,2.0,33.33,0.0,0.00,0.0,0.00,1.0,...,0.0,0.0,4.00,1.55,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
5,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3.0,50.00,2.0,33.33,1.0,16.67,0.0,0.00,0.0,...,0.0,0.0,4.33,0.82,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ผู้สอนแทรกเนื้อหาเกี่ยวกับคุณธรรมจริยธรรมในการ...,15.0,51.72,9.0,31.03,3.0,10.34,2.0,6.90,0.0,...,0.0,0.0,4.28,0.92,ส่วนที่ 2 ข้อคำถามของคณะ/หน่วยงาน,231863,NYE553,1/2563,43,29 (67.44%)
1,โสตทัศนูปกรณ์ช่วยสอนในห้องเรียน เช่น ไมโครโฟน ...,14.0,48.28,8.0,27.59,5.0,17.24,1.0,3.45,1.0,...,0.0,0.0,4.14,1.06,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,231863,NYE553,1/2563,43,29 (67.44%)
2,สภาพของห้องเรียนหรือห้องปฏิบัติการ เช่น แสง อุ...,13.0,44.83,11.0,37.93,4.0,13.79,0.0,0.00,1.0,...,0.0,0.0,4.21,0.94,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,231863,NYE553,1/2563,43,29 (67.44%)


In [ ]:
df.info() #ใช้คำสั่ง df.info() เพื่อตรวจสอบ Data Type

<class 'pandas.core.frame.DataFrame'>
Index: 324 entries, 1 to 4
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   ข้อคำถาม                   324 non-null    object 
 1   5                          324 non-null    float64
 2   5(%)                       324 non-null    float64
 3   4                          324 non-null    float64
 4   4(%)                       324 non-null    float64
 5   3                          324 non-null    float64
 6   3(%)                       324 non-null    float64
 7   2                          324 non-null    float64
 8   2(%)                       324 non-null    float64
 9   1                          324 non-null    float64
 10  1(%)                       324 non-null    float64
 11  0                          324 non-null    float64
 12  0(%)                       324 non-null    float64
 13  X-Bar                      324 non-null    float64
 14  S

In [ ]:
#ดึงข้อมูลทั้งหมดเข้า excel เพื่อช่วยให้เห็นภาพรวมของข้อมูล
df.to_excel("projectDSI200.xlsx", encoding = "utf-8")

In [ ]:
#การเลือกใช้ข้อมูลที่ต้องการเพื่อนำมาแสดงเป็น Data Frame เช่น
df[(df["รายวิชา"] == 'FGL497') & (df["section"] == '231863')]
#df[(df["รายวิชา"] == 'NYE553')& (df["section"] == '231863') | (df['ภาคการศึกษา'] == '1/2564')]

,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,0,0(%),X-Bar,SD,Topic,section,รายวิชา,ภาคการศึกษา,จำนวนนักศึกษาที่จดทะเบียน,จำนวนนักศึกษาที่ประเมิน
ที่,,,,,,,,,,,,,,,,,,,,,
1,สอนอย่างเป็นระบบ,4.0,66.67,1.0,16.67,0.0,0.00,1.0,16.67,0.0,...,0.0,0.0,4.33,1.21,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
2,สอนให้คิด วิเคราะห์ วิจารณ์,4.0,66.67,0.0,0.00,1.0,16.67,1.0,16.67,0.0,...,0.0,0.0,4.17,1.33,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
3,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2.0,33.33,1.0,16.67,1.0,16.67,1.0,16.67,1.0,...,0.0,0.0,3.33,1.63,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
4,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3.0,50.00,2.0,33.33,0.0,0.00,0.0,0.00,1.0,...,0.0,0.0,4.00,1.55,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
5,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3.0,50.00,2.0,33.33,1.0,16.67,0.0,0.00,0.0,...,0.0,0.0,4.33,0.82,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
6,ทำให้เห็นความสัมพันธ์กับวิชาอื่นที่เกี่ยวข้อง,3.0,50.00,2.0,33.33,0.0,0.00,1.0,16.67,0.0,...,0.0,0.0,4.17,1.17,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
7,ใช้สื่อและอุปกรณ์ช่วยสอนได้ดี,3.0,50.00,2.0,33.33,0.0,0.00,1.0,16.67,0.0,...,0.0,0.0,4.17,1.17,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
8,แนะนำแหล่งค้นคว้าข้อมูลเพิ่มเติมให้,3.0,50.00,1.0,16.67,1.0,16.67,1.0,16.67,0.0,...,0.0,0.0,4.00,1.26,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1/2562,43,6 (13.95%)
1,ผู้สอนแจ้งวัตถุประสงค์และเนื้อหารายวิชาตามเค้า...,4.0,66.67,1.0,16.67,1.0,16.67,0.0,0.00,0.0,...,0.0,0.0,4.50,0.84,ส่วนที่ 2 ข้อคำถามของคณะ/หน่วยงาน,231863,FGL497,1/2562,43,6 (13.95%)


สร้างกราฟ sunburst ด้วยคำสั่ง px.sunburst เพื่อแสดงผลโดยรวมของจำนวนนักศึกษาที่จดทะเบียนและจำนวนนักศึกษาที่ประเมินทุกวิชาและทุก section ในปี 2561-2564

In [ ]:
fig = px.sunburst(df,path=["ภาคการศึกษา","รายวิชา","section","จำนวนนักศึกษาที่จดทะเบียน","จำนวนนักศึกษาที่ประเมิน"],
                  title = "The number of registed students and evaluated students",template="plotly_dark")
fig.show()

In [ ]:
df[["จำนวนนักศึกษาที่ประเมิน","ร้อยละจำนวนนักศึกษาที่ประเมิน"]] = df["จำนวนนักศึกษาที่ประเมิน"].str.split("(", expand=True) #แบ่งคอลัมน์จำนวนนักศึกษาที่ประเมิน จากสัญลักษณ์ ()
df[["ภาคการศึกษา","ปีการศึกษา"]] = df["ภาคการศึกษา"].str.split("/", expand=True) #แบ่งคอลัมน์ภาคการศึกษาเป็น 2 คอลัมน์ ด้วยสัญลักษณ์ ()

df

,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,X-Bar,SD,Topic,section,รายวิชา,ภาคการศึกษา,จำนวนนักศึกษาที่จดทะเบียน,จำนวนนักศึกษาที่ประเมิน,ร้อยละจำนวนนักศึกษาที่ประเมิน,ปีการศึกษา
ที่,,,,,,,,,,,,,,,,,,,,,
1,สอนอย่างเป็นระบบ,4.0,66.67,1.0,16.67,0.0,0.00,1.0,16.67,0.0,...,4.33,1.21,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43,6,13.95%),2562
2,สอนให้คิด วิเคราะห์ วิจารณ์,4.0,66.67,0.0,0.00,1.0,16.67,1.0,16.67,0.0,...,4.17,1.33,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43,6,13.95%),2562
3,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2.0,33.33,1.0,16.67,1.0,16.67,1.0,16.67,1.0,...,3.33,1.63,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43,6,13.95%),2562
4,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3.0,50.00,2.0,33.33,0.0,0.00,0.0,0.00,1.0,...,4.00,1.55,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43,6,13.95%),2562
5,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3.0,50.00,2.0,33.33,1.0,16.67,0.0,0.00,0.0,...,4.33,0.82,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43,6,13.95%),2562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ผู้สอนแทรกเนื้อหาเกี่ยวกับคุณธรรมจริยธรรมในการ...,15.0,51.72,9.0,31.03,3.0,10.34,2.0,6.90,0.0,...,4.28,0.92,ส่วนที่ 2 ข้อคำถามของคณะ/หน่วยงาน,231863,NYE553,1,43,29,67.44%),2563
1,โสตทัศนูปกรณ์ช่วยสอนในห้องเรียน เช่น ไมโครโฟน ...,14.0,48.28,8.0,27.59,5.0,17.24,1.0,3.45,1.0,...,4.14,1.06,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,231863,NYE553,1,43,29,67.44%),2563
2,สภาพของห้องเรียนหรือห้องปฏิบัติการ เช่น แสง อุ...,13.0,44.83,11.0,37.93,4.0,13.79,0.0,0.00,1.0,...,4.21,0.94,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,231863,NYE553,1,43,29,67.44%),2563


In [ ]:
df["ร้อยละจำนวนนักศึกษาที่ประเมิน"] = df["ร้อยละจำนวนนักศึกษาที่ประเมิน"].str.replace('%','') #แทนที่ % ด้วยช่องว่าง
df["ร้อยละจำนวนนักศึกษาที่ประเมิน"] = df["ร้อยละจำนวนนักศึกษาที่ประเมิน"].str.replace(')','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



In [ ]:
#เปลี่ยน data type เป็น float และ int
df['ร้อยละจำนวนนักศึกษาที่ประเมิน']= df['ร้อยละจำนวนนักศึกษาที่ประเมิน'].astype('float')
df['จำนวนนักศึกษาที่ประเมิน']= df['จำนวนนักศึกษาที่ประเมิน'].astype('float')
df['จำนวนนักศึกษาที่จดทะเบียน']= df['จำนวนนักศึกษาที่จดทะเบียน'].astype('float')
df['ปีการศึกษา']= df['ปีการศึกษา'].astype('int')

In [ ]:
df.info() #ใช้คำสั่ง df.info() เพื่อตรวจสอบ Data Type

<class 'pandas.core.frame.DataFrame'>
Index: 324 entries, 1 to 4
Data columns (total 23 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ข้อคำถาม                       324 non-null    object 
 1   5                              324 non-null    float64
 2   5(%)                           324 non-null    float64
 3   4                              324 non-null    float64
 4   4(%)                           324 non-null    float64
 5   3                              324 non-null    float64
 6   3(%)                           324 non-null    float64
 7   2                              324 non-null    float64
 8   2(%)                           324 non-null    float64
 9   1                              324 non-null    float64
 10  1(%)                           324 non-null    float64
 11  0                              324 non-null    float64
 12  0(%)                           324 non-null    float64
 1

In [ ]:
df['ปีการศึกษา'].unique() #หาค่าที่ไม่ซ้ำกันของคอลัมน์ 'ปีการศึกษา'

array([2562, 2563, 2564, 2561])

In [ ]:
#สร้างฟังก์ชั่นเลือกเฉพาะข้อมูลปีที่มี covid
df1 = df.loc[df['ปีการศึกษา'] > 2561]
df1

,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,X-Bar,SD,Topic,section,รายวิชา,ภาคการศึกษา,จำนวนนักศึกษาที่จดทะเบียน,จำนวนนักศึกษาที่ประเมิน,ร้อยละจำนวนนักศึกษาที่ประเมิน,ปีการศึกษา
ที่,,,,,,,,,,,,,,,,,,,,,
1,สอนอย่างเป็นระบบ,4.0,66.67,1.0,16.67,0.0,0.00,1.0,16.67,0.0,...,4.33,1.21,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
2,สอนให้คิด วิเคราะห์ วิจารณ์,4.0,66.67,0.0,0.00,1.0,16.67,1.0,16.67,0.0,...,4.17,1.33,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
3,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2.0,33.33,1.0,16.67,1.0,16.67,1.0,16.67,1.0,...,3.33,1.63,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
4,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3.0,50.00,2.0,33.33,0.0,0.00,0.0,0.00,1.0,...,4.00,1.55,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
5,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3.0,50.00,2.0,33.33,1.0,16.67,0.0,0.00,0.0,...,4.33,0.82,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ผู้สอนแทรกเนื้อหาเกี่ยวกับคุณธรรมจริยธรรมในการ...,15.0,51.72,9.0,31.03,3.0,10.34,2.0,6.90,0.0,...,4.28,0.92,ส่วนที่ 2 ข้อคำถามของคณะ/หน่วยงาน,231863,NYE553,1,43.0,29.0,67.44,2563
1,โสตทัศนูปกรณ์ช่วยสอนในห้องเรียน เช่น ไมโครโฟน ...,14.0,48.28,8.0,27.59,5.0,17.24,1.0,3.45,1.0,...,4.14,1.06,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,231863,NYE553,1,43.0,29.0,67.44,2563
2,สภาพของห้องเรียนหรือห้องปฏิบัติการ เช่น แสง อุ...,13.0,44.83,11.0,37.93,4.0,13.79,0.0,0.00,1.0,...,4.21,0.94,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,231863,NYE553,1,43.0,29.0,67.44,2563


In [ ]:
df11 =df1.groupby('ปีการศึกษา').sum() #หาผลรวมทุกปีการศึกษา
df11

,5,5(%),4,4(%),3,3(%),2,2(%),1,1(%),0,0(%),X-Bar,SD,จำนวนนักศึกษาที่จดทะเบียน,จำนวนนักศึกษาที่ประเมิน,ร้อยละจำนวนนักศึกษาที่ประเมิน
ปีการศึกษา,,,,,,,,,,,,,,,,,
2562,1148.0,4044.59,1012.0,2767.70,650.0,1350.29,274.0,397.66,162.0,202.06,74.0,37.73,364.76,78.09,12600.0,3328.0,2392.92
2563,2071.0,7091.14,877.0,3698.75,370.0,1441.35,66.0,297.74,36.0,225.95,16.0,45.00,555.99,104.96,9772.0,3504.0,5017.52
2564,1029.0,3831.32,431.0,1225.42,165.0,415.13,39.0,109.43,7.0,16.10,1.0,2.50,255.56,33.11,3416.0,1704.0,2642.48


In [ ]:
df2 =df1.groupby(['ปีการศึกษา','รายวิชา'])['รายวิชา'].count() #รวมจำนวนรายวิชาในแต่ละปีการศึกษา
df2 = pd.DataFrame(df2) #สร้าง data frame จากตัวแปร df2
df2

รายวิชา
ปีการศึกษา รายวิชา         
2562       DHT256        20
           FGL497        28
           RMW014        20
           YRY790        20
2563       AZW892         8
           DRI970        40
           FGL497        24
           NVI781         8
           NYE553        20
           RMW014         8
           YRY790        20
2564       AZW892         8
           DRI970         8
           FGL497         8
           NVI781         8
           NYE553         8
           XCM602         8
           YRY790         8

In [ ]:
result=pd.merge(df11, df2, on="ปีการศึกษา") #รวม dataframe โดยอิงจากปีการศึกษา
result

,5,5(%),4,4(%),3,3(%),2,2(%),1,1(%),0,0(%),X-Bar,SD,จำนวนนักศึกษาที่จดทะเบียน,จำนวนนักศึกษาที่ประเมิน,ร้อยละจำนวนนักศึกษาที่ประเมิน,รายวิชา
ปีการศึกษา,,,,,,,,,,,,,,,,,,
2562,1148.0,4044.59,1012.0,2767.70,650.0,1350.29,274.0,397.66,162.0,202.06,74.0,37.73,364.76,78.09,12600.0,3328.0,2392.92,20
2562,1148.0,4044.59,1012.0,2767.70,650.0,1350.29,274.0,397.66,162.0,202.06,74.0,37.73,364.76,78.09,12600.0,3328.0,2392.92,28
2562,1148.0,4044.59,1012.0,2767.70,650.0,1350.29,274.0,397.66,162.0,202.06,74.0,37.73,364.76,78.09,12600.0,3328.0,2392.92,20
2562,1148.0,4044.59,1012.0,2767.70,650.0,1350.29,274.0,397.66,162.0,202.06,74.0,37.73,364.76,78.09,12600.0,3328.0,2392.92,20
2563,2071.0,7091.14,877.0,3698.75,370.0,1441.35,66.0,297.74,36.0,225.95,16.0,45.00,555.99,104.96,9772.0,3504.0,5017.52,8
2563,2071.0,7091.14,877.0,3698.75,370.0,1441.35,66.0,297.74,36.0,225.95,16.0,45.00,555.99,104.96,9772.0,3504.0,5017.52,40
2563,2071.0,7091.14,877.0,3698.75,370.0,1441.35,66.0,297.74,36.0,225.95,16.0,45.00,555.99,104.96,9772.0,3504.0,5017.52,24
2563,2071.0,7091.14,877.0,3698.75,370.0,1441.35,66.0,297.74,36.0,225.95,16.0,45.00,555.99,104.96,9772.0,3504.0,5017.52,8
2563,2071.0,7091.14,877.0,3698.75,370.0,1441.35,66.0,297.74,36.0,225.95,16.0,45.00,555.99,104.96,9772.0,3504.0,5017.52,20


In [ ]:
result1 = result['X-Bar']/result['รายวิชา'] #คำนวณเพื่อเอามาวาดกราฟ

In [ ]:
result1 = pd.DataFrame(result1).reset_index() #สร้างคอลัมน์

In [ ]:
result1['ปีการศึกษา'] = result1['ปีการศึกษา'].astype(str)

In [ ]:
result1.set_axis(['ปีการศึกษา', 'คะแนน'], axis='columns', inplace=True) #ตั้งชื่อคอลัมน์
result1

,ปีการศึกษา,คะแนน
0,2562,18.238000
1,2562,13.027143
2,2562,18.238000
3,2562,18.238000
4,2563,69.498750
5,2563,13.899750
6,2563,23.166250
7,2563,69.498750
8,2563,27.799500
9,2563,69.498750


สร้างกราฟด้วยคำสั่ง px.bar เพื่อแสดง คะแนนรวมของทุกวิชาในปีการศึกษา 2562-2564 (ช่วง Covid-19)

In [ ]:
px.bar(result1, x="ปีการศึกษา",y="คะแนน",
           title = "คะแนนรวมของทุกวิชาในปีการศึกษา 2562-2564 (ช่วง Covid-19)")

สร้าง data frame เพื่อนำไปใช้ในการสร้างกราฟ แสดงค่า max min mean ของแต่ละรายวิชา

In [ ]:
df1 = df.groupby('รายวิชา')["ร้อยละจำนวนนักศึกษาที่ประเมิน"].count()\

In [ ]:
df2 =pd.DataFrame(df1) #สร้าง data frame df2 จาก df1
df2

,ร้อยละจำนวนนักศึกษาที่ประเมิน
รายวิชา,
AZW892,16
DHT256,20
DRI970,48
FGL497,60
NVI781,36
NYE553,28
RMW014,40
XCM602,8
YRY790,68


In [ ]:
df2 =df2.reset_index()
df2

,รายวิชา,ร้อยละจำนวนนักศึกษาที่ประเมิน
0,AZW892,16
1,DHT256,20
2,DRI970,48
3,FGL497,60
4,NVI781,36
5,NYE553,28
6,RMW014,40
7,XCM602,8
8,YRY790,68


In [ ]:
df #เรียกดูข้อมูลจาก df

,ข้อคำถาม,5,5(%),4,4(%),3,3(%),2,2(%),1,...,X-Bar,SD,Topic,section,รายวิชา,ภาคการศึกษา,จำนวนนักศึกษาที่จดทะเบียน,จำนวนนักศึกษาที่ประเมิน,ร้อยละจำนวนนักศึกษาที่ประเมิน,ปีการศึกษา
ที่,,,,,,,,,,,,,,,,,,,,,
1,สอนอย่างเป็นระบบ,4.0,66.67,1.0,16.67,0.0,0.00,1.0,16.67,0.0,...,4.33,1.21,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
2,สอนให้คิด วิเคราะห์ วิจารณ์,4.0,66.67,0.0,0.00,1.0,16.67,1.0,16.67,0.0,...,4.17,1.33,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
3,กระตุ้นบรรยากาศการเรียนการสอน มีวิธีสอนที่น่าส...,2.0,33.33,1.0,16.67,1.0,16.67,1.0,16.67,1.0,...,3.33,1.63,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
4,เปิดโอกาสให้แสดงความคิดเห็นและแลกเปลี่ยนประสบก...,3.0,50.00,2.0,33.33,0.0,0.00,0.0,0.00,1.0,...,4.00,1.55,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
5,มีการให้นำเสนองาน/มีการวัดผลโดยวิธีอื่นๆที่สะท...,3.0,50.00,2.0,33.33,1.0,16.67,0.0,0.00,0.0,...,4.33,0.82,ส่วนที่ 1 ข้อคำถามกลางของมหาวิทยาลัย,231863,FGL497,1,43.0,6.0,13.95,2562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8,ผู้สอนแทรกเนื้อหาเกี่ยวกับคุณธรรมจริยธรรมในการ...,15.0,51.72,9.0,31.03,3.0,10.34,2.0,6.90,0.0,...,4.28,0.92,ส่วนที่ 2 ข้อคำถามของคณะ/หน่วยงาน,231863,NYE553,1,43.0,29.0,67.44,2563
1,โสตทัศนูปกรณ์ช่วยสอนในห้องเรียน เช่น ไมโครโฟน ...,14.0,48.28,8.0,27.59,5.0,17.24,1.0,3.45,1.0,...,4.14,1.06,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,231863,NYE553,1,43.0,29.0,67.44,2563
2,สภาพของห้องเรียนหรือห้องปฏิบัติการ เช่น แสง อุ...,13.0,44.83,11.0,37.93,4.0,13.79,0.0,0.00,1.0,...,4.21,0.94,ส่วนที่ 3 สภาพแวดล้อมและสิ่งสนับสนุนการเรียนรู้,231863,NYE553,1,43.0,29.0,67.44,2563


In [ ]:
df3 = df.groupby('รายวิชา')["X-Bar"].describe() #แบ่งกลุ่มข้อมูลค่า max min mean ของแต่ละรายวิชา
df3

,count,mean,std,min,25%,50%,75%,max
รายวิชา,,,,,,,,
AZW892,16.0,4.540625,0.297152,3.80,4.4000,4.605,4.7675,4.88
DHT256,20.0,4.091500,0.326727,3.50,3.8300,4.170,4.3300,4.67
DRI970,48.0,4.487292,0.230363,3.92,4.3150,4.500,4.6475,4.96
FGL497,60.0,4.289667,0.374813,3.17,4.0000,4.330,4.5000,5.00
NVI781,36.0,4.512222,0.250098,3.43,4.4175,4.570,4.6875,4.86
NYE553,28.0,4.472857,0.256787,4.03,4.2800,4.515,4.7100,4.93
RMW014,40.0,4.074750,0.400480,3.22,3.7575,4.165,4.3775,4.63
XCM602,8.0,5.000000,0.000000,5.00,5.0000,5.000,5.0000,5.00
YRY790,68.0,4.093088,0.460491,2.88,3.8325,4.130,4.4475,4.86


In [ ]:
df3 =df3.reset_index()  #ทำ df3 ให้เป็นคอลัมน์
df3

,รายวิชา,count,mean,std,min,25%,50%,75%,max
0,AZW892,16.0,4.540625,0.297152,3.80,4.4000,4.605,4.7675,4.88
1,DHT256,20.0,4.091500,0.326727,3.50,3.8300,4.170,4.3300,4.67
2,DRI970,48.0,4.487292,0.230363,3.92,4.3150,4.500,4.6475,4.96
3,FGL497,60.0,4.289667,0.374813,3.17,4.0000,4.330,4.5000,5.00
4,NVI781,36.0,4.512222,0.250098,3.43,4.4175,4.570,4.6875,4.86
5,NYE553,28.0,4.472857,0.256787,4.03,4.2800,4.515,4.7100,4.93
6,RMW014,40.0,4.074750,0.400480,3.22,3.7575,4.165,4.3775,4.63
7,XCM602,8.0,5.000000,0.000000,5.00,5.0000,5.000,5.0000,5.00
8,YRY790,68.0,4.093088,0.460491,2.88,3.8325,4.130,4.4475,4.86


สร้างกราฟด้วยคำสั่ง  px.bar_polar เพื่อเปรียบเทียบค่า max และ min ทุกรายวิชา ในปี 2561-2564

In [ ]:
fig = px.bar_polar(df3, r="min", theta="รายวิชา",color="max",
                   template="plotly_dark", title = "ค่า Max Min ของแต่ละรายวิชา")
fig.show()

สร้างกราฟด้วยคำสั่ง px.line เพื่อแสดงค่า min ของแต่ละรายวิชา ในปี 2561-2564

In [ ]:
fig = px.line(df3, x="รายวิชา", y="min",
              title = "ค่า min ของแต่ละรายวิชา")
fig.show()

สร้างกราฟด้วยคำสั่ง px.line เพื่อแสดงค่า max ของแต่ละรายวิชา ในปี 2561-2564

In [ ]:
fig = px.line(df3, x="รายวิชา", y="max",
              title = "ค่า max ของแต่ละรายวิชา")
fig.show()

สร้างกราฟด้วยคำสั่ง px.line เพื่อแสดงค่า mean ของแต่ละรายวิชา ในปี 2561-2564

In [ ]:
fig = px.line(df3, x="รายวิชา", y="mean",
              title = "ค่า mean ของแต่ละรายวิชา")
fig.show()

#บทสรุป

การ Scraping  คือการดึงข้อมูลจากเว็บไซต์มาใช้ประโยชน์ เพื่อนำข้อมูลมาวิเคราะห์  หาเหตุผล หรือ Insight บางอย่างที่จะได้จากข้อมูลบนเว็บไซต์ 
Data Visualization คือการนำเสนอข้อมูลในรูปแบบภาพนิ่ง เพื่อให้สามารถเข้าใจข้อมูลได้ง่ายขึ้น ซึ่งกราฟที่เราได้จากการ Scraping ข้อมูลการประเมินรายวิชาจากนักศึกษา กลุ่มพวกเราได้นำมาทำกราฟหลากหลายรูปแบบเพื่อวิเคราะห์ข้อมูล โดยกราฟที่เราทำการ Visualization ทำให้เพวกเราได้ทราบข้อมูลโดยรวมทั้งหมด ทราบค่าคะแนนการประเมิน Max Min Mean ของแต่ละวิชา เพื่อสามารถนำไปปรับใช้ในการสอนในแต่ละวิชาต่อไป 